In [17]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset,DataLoader 


In [18]:
# setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_df = pd.read_csv("data/fashion-mnist_train.csv")
test_data_df = pd.read_csv("data/fashion-mnist_test.csv")
train_data_df


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# model

class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        

        self.fc_layer = nn.Sequential(
            nn.Linear(128*3*3, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = x.view(-1,128*3*3)
        x = self.fc_layer(x)

        return x


In [20]:
# custom dataset
class FashionMNISTDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data : pd.DataFrame = dataframe
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):

        # start from index 1, first one is the label int8 is enough cause 0 to 255
        image = self.data.iloc[idx, 1:].values.astype(np.uint8).reshape(28, 28, 1)
        
        label = self.data.iloc[idx, 0]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label


In [21]:
# load the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNISTDataset(train_data_df,transform=transform)
test_dataset = FashionMNISTDataset(test_data_df,transform=transform)

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# setup 
model = FashionCNN().to(device)
criterion : nn .CrossEntropyLoss = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)


In [ ]:
#training the model

num_epochs = 10 

for epoch in range(num_epochs):

    model.train()
    running_loss = 0.0

    # correct in training
    correct = 0
    total = 0
    
    # retreive a batch of images and labels and feed them to the device
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss  = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()

        # compute the new gradients, and then update the weights
        loss.backward()
        optimizer.step()
        
        #this is the total loss for the entire train set
        running_loss += loss.item()

        #get the max value of the softmax function second item of tuple is the index, the first one is the value
        _, predicted_index = torch.max(outputs.data, 1) 
        total += labels.size(0)
        correct += (predicted_index == labels).sum().item()
    
    # Print statistics
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    # Validation mode
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    # this temporarily disables the autograd engine for faster computations
    # no backward() call so 
    with torch.no_grad():

        for images, labels in test_loader:

            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(test_loader)
    val_acc = 100 * val_correct / val_total
    
    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

    scheduler.step(val_acc)

Epoch [1/10], Train Loss: 0.5543, Train Acc: 79.65% 
Epoch [2/10], Train Loss: 0.3607, Train Acc: 86.87% 
Epoch [3/10], Train Loss: 0.3229, Train Acc: 88.31% 
Epoch [4/10], Train Loss: 0.2950, Train Acc: 89.17% 
Epoch [5/10], Train Loss: 0.2848, Train Acc: 89.56% 
Epoch [6/10], Train Loss: 0.2683, Train Acc: 90.17% 
Epoch [7/10], Train Loss: 0.2609, Train Acc: 90.35% 
Epoch [8/10], Train Loss: 0.2476, Train Acc: 90.91% 
Epoch [9/10], Train Loss: 0.2428, Train Acc: 91.06% 
Epoch [10/10], Train Loss: 0.2361, Train Acc: 91.23% 


In [24]:
# save the model parameters
torch.save(model.state_dict(), "fashion_cnn.pth")

In [25]:
# testing the model with the test set now use eval mode for faster computation
model.eval()

test_loss = 0
correct = 0
total = 0

 # this temporarily disables the autograd engine for faster computations
    # no backward() call so 
with torch.no_grad():

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # orward pass
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        # Get predictions
        _, predicted = torch.max(outputs.data, 1)
        
        # Update statistics
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct / total

print(f'\nTest Results:')
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Results:
Test Loss: 0.2074
Test Accuracy: 92.18%
